In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import Workbook, load_workbook
import os
from numpy import random
import pycytominer
import utilitary
import plotly.express as px

# Inputs

In [40]:
file_name= r"G:\My Drive\2022_Garcia-Fossa_submitted\basic_protocol_1\notebooks\Morpheus_Example_FeatureSelected.csv"

## DataFrame from a csv file

In [25]:
#open DataFrame
df = pd.read_csv(file_name, low_memory=False)
df.drop(df.columns[0], axis = 'columns', inplace=True)

In [28]:
df

,Metadata_Concentration,Metadata_moa,Metadata_Compound,Metadata_Plate,Metadata_Well,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,Cells_AreaShape_Orientation,...,Nuclei_Texture_SumAverage_DNA_5_0,Nuclei_Texture_SumAverage_RNA_20_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumEntropy_RNA_20_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_Variance_DNA_20_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_RNA_20_0
0,0.00,DMSO,DMSO,SQ00015195,A01,-0.822020,-0.075336,1.223834,-1.956320,1.265650,...,-1.440937,-1.009833,1.300275,0.898846,1.484104,-0.771877,-0.233147,1.673231,1.411891,1.526507
1,0.00,DMSO,DMSO,SQ00015195,A02,0.868327,2.649399,0.682239,-0.680453,0.023257,...,-3.206421,-4.473190,-0.102291,-0.480672,-0.350701,-4.866834,-3.849305,-0.368757,0.682114,0.381323
2,0.00,DMSO,DMSO,SQ00015195,A03,-1.053129,-1.248110,0.666743,-1.064470,-0.212970,...,0.121409,-0.811745,0.222379,0.277556,0.166181,-0.678615,-0.396402,0.160436,-1.030875,0.071883
3,0.00,DMSO,DMSO,SQ00015195,A04,-0.907778,-0.629663,2.049594,0.306481,-0.338950,...,0.653975,2.015985,0.914372,0.562561,1.167740,1.179171,1.419980,1.147498,2.560806,0.771792
4,0.00,DMSO,DMSO,SQ00015195,A05,-0.708257,-0.422551,-0.115723,-0.524645,1.027344,...,-0.050480,1.102265,1.040591,1.113570,1.084811,0.349723,0.061666,1.209480,1.142756,-1.148015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,3.33,topoisomerase inhibitor,etoposide,SQ00015221,P20,-2.399107,-0.821501,6.260044,-7.330029,-3.563681,...,0.928834,-2.172056,-2.668529,-1.349427,-8.328093,-2.165124,-1.894194,-3.714304,0.285391,-0.825984
1916,1.11,topoisomerase inhibitor,etoposide,SQ00015221,P21,-3.331789,-0.797576,4.358127,-5.007127,-0.988478,...,-1.846109,-0.203125,1.950478,3.023224,-0.566485,0.222524,-1.158631,1.646859,2.064042,-0.736931
1917,0.37,topoisomerase inhibitor,etoposide,SQ00015221,P22,-2.196081,-1.589165,2.453490,-1.774681,-0.730178,...,-0.684405,-0.963087,1.192883,1.268240,1.498845,-2.343231,-1.452264,0.569056,0.653778,-1.290920
1918,0.12,topoisomerase inhibitor,etoposide,SQ00015221,P23,-2.605366,-2.723491,2.006497,-1.898010,-0.281345,...,-0.339666,-1.515713,0.321631,0.537636,0.999894,-3.737373,-2.364335,-0.300792,-0.159041,-1.883261


In [29]:
replicability_ap_df = pd.DataFrame()
matching_ap_df = pd.DataFrame()

In [31]:
#add metadata_control_type column
all_plates_df = df.copy()
all_plates_df['Metadata_control_type'] = ''
cmpd = all_plates_df['Metadata_Compound'].values
ctrl = all_plates_df['Metadata_control_type'].values
for vals in range(len(cmpd)):
    if cmpd[vals] == "DMSO":
        ctrl[vals] = 'negcon'

In [32]:
all_plates_df

,Metadata_Concentration,Metadata_moa,Metadata_Compound,Metadata_Plate,Metadata_Well,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_Extent,Cells_AreaShape_FormFactor,Cells_AreaShape_Orientation,...,Nuclei_Texture_SumAverage_RNA_20_0,Nuclei_Texture_SumEntropy_DNA_10_0,Nuclei_Texture_SumEntropy_DNA_20_0,Nuclei_Texture_SumEntropy_DNA_5_0,Nuclei_Texture_SumEntropy_RNA_20_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_Variance_DNA_20_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_RNA_20_0,Metadata_control_type
0,0.00,DMSO,DMSO,SQ00015195,A01,-0.822020,-0.075336,1.223834,-1.956320,1.265650,...,-1.009833,1.300275,0.898846,1.484104,-0.771877,-0.233147,1.673231,1.411891,1.526507,negcon
1,0.00,DMSO,DMSO,SQ00015195,A02,0.868327,2.649399,0.682239,-0.680453,0.023257,...,-4.473190,-0.102291,-0.480672,-0.350701,-4.866834,-3.849305,-0.368757,0.682114,0.381323,negcon
2,0.00,DMSO,DMSO,SQ00015195,A03,-1.053129,-1.248110,0.666743,-1.064470,-0.212970,...,-0.811745,0.222379,0.277556,0.166181,-0.678615,-0.396402,0.160436,-1.030875,0.071883,negcon
3,0.00,DMSO,DMSO,SQ00015195,A04,-0.907778,-0.629663,2.049594,0.306481,-0.338950,...,2.015985,0.914372,0.562561,1.167740,1.179171,1.419980,1.147498,2.560806,0.771792,negcon
4,0.00,DMSO,DMSO,SQ00015195,A05,-0.708257,-0.422551,-0.115723,-0.524645,1.027344,...,1.102265,1.040591,1.113570,1.084811,0.349723,0.061666,1.209480,1.142756,-1.148015,negcon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,3.33,topoisomerase inhibitor,etoposide,SQ00015221,P20,-2.399107,-0.821501,6.260044,-7.330029,-3.563681,...,-2.172056,-2.668529,-1.349427,-8.328093,-2.165124,-1.894194,-3.714304,0.285391,-0.825984,
1916,1.11,topoisomerase inhibitor,etoposide,SQ00015221,P21,-3.331789,-0.797576,4.358127,-5.007127,-0.988478,...,-0.203125,1.950478,3.023224,-0.566485,0.222524,-1.158631,1.646859,2.064042,-0.736931,
1917,0.37,topoisomerase inhibitor,etoposide,SQ00015221,P22,-2.196081,-1.589165,2.453490,-1.774681,-0.730178,...,-0.963087,1.192883,1.268240,1.498845,-2.343231,-1.452264,0.569056,0.653778,-1.290920,
1918,0.12,topoisomerase inhibitor,etoposide,SQ00015221,P23,-2.605366,-2.723491,2.006497,-1.898010,-0.281345,...,-1.515713,0.321631,0.537636,0.999894,-3.737373,-2.364335,-0.300792,-0.159041,-1.883261,


In [33]:
feature_to_group_by = 'Metadata_Compound'
# Description
description = f'compound plate'

# Calculate replicability mAP
print(f'Computing {description} replicability')
precision = utilitary.PrecisionScores(all_plates_df, all_plates_df, feature_to_group_by, "replicability", feature_to_group_by, within=True, against_negcon=True)

Computing compound plate replicability


In [34]:
replicability_ap_df = precision.ap_group
replicability_map = precision.map

In [35]:
replicability_ap_df

,Metadata_Compound,ap
0,AGI-6780,0.312469
1,AZ-628,0.411881
2,BMS-599626,0.328812
3,CPI-613,0.200995
4,LY2334737,0.404135
5,MG-132,0.804993
6,PD-184352,0.632147
7,PF-04691502,0.599414
8,PSI-6130,0.369457
9,PSI-7976,0.128603


In [36]:
replicability_map

0.37246913867686876

In [42]:
replicable_compounds = list(replicability_ap_df[replicability_ap_df.ap>0][feature_to_group_by])
non_replicable_compounds = list(replicability_ap_df[replicability_ap_df.ap<0][feature_to_group_by])

In [44]:
print("Number of replicable compounds", len(replicable_compounds))
print("Number of NOT replicable compounds", len(non_replicable_compounds))

Number of replicable compounds 58
Number of NOT replicable compounds 0
